**Import the useful packages** 

In [1]:
import numpy as np
import xarray as xr
import glob
import matplotlib.pyplot as plt
import os,sys
import pandas as pd
import itertools

In [2]:
from functions import load_test_data_whole_sim, load_model_nn, load_normalisation_metrics, regression_analysis, load_results_nn, \
ticks_labels, plot_monthly_trend, RMSE_R2, std_range, twosigfig

**Set the filepaths to the input and output data** 

In [3]:
path_model = '/bettik/ockendeh/SCRIPTS/simpleNN_basal_melt/AIAI_data/NN_models/'
fp_fdata = 'Figure_data/'
#path_norm_metrics = '/bettik/ockendeh/SCRIPTS/simpleNN_basal_melt/AIAI_data/Training_data/'

#models = glob.glob(path_model + '*.keras')
#histories = glob.glob(path_model + 'history*.csv')

#fp_all_data = '/bettik/ockendeh/SCRIPTS/simpleNN_basal_melt/AIAI_data/Training_data/whole_dataset_not_yet_normalised.csv'


**Look at the results of the Neural Network**

In [4]:
# Check to see which combinations are actually available...
fp_ref_pred = glob.glob(path_model + 'applied_nn' + '_small_slope_front_' + '*' + '_' + '' + \
                                        '*' + '_extrap_std' + '*' + '.csv')
for i in range(len(fp_ref_pred)):
    simulations = fp_ref_pred[i].split('_small_slope_front_')[1].split('.csv')[0].split('_all_sims_extrap_std_special_lc_')
    print(fp_ref_pred[i].split('_small_slope_front_')[1].split('.csv')[0].split('_all_sims_extrap_std_special_lc_'))

['OPM031_40p0', 'Christoph_v2_whole_dataset']
['OPM026_OPM0263_OPM031_OPM016_OPM018_OPM021_ctrl94_isf94_isfru94', 'OPM031_whole_dataset']
['OPM031_0p40', 'Christoph_v2_whole_dataset']
['OPM0263', 'OPM0263_whole_dataset']
['OPM031_0p80', 'Christoph_v2_whole_dataset']
['after_1984_annual_all_sims_extrap_std_annual_special_lc_whole_dataset']
['OPM031_10p10', 'Christoph_v2_whole_dataset']
['OPM026_annual', 'OPM031_whole_dataset']
['OPM031_annual', 'Christoph_v2_whole_dataset']
['OPM026_OPM0263_OPM031_OPM016_OPM018_OPM021_ctrl94_isf94_isfru94', 'ctrl94_whole_dataset']
['OPM026_to2018', 'OPM0263_whole_dataset']
['OPM026_to1998', 'OPM0263_whole_dataset']
['no_mar_oct_annual_all_sims_extrap_std_annual_special_lc_whole_dataset']
['OPM031_0p2', 'Christoph_v2_whole_dataset']
['OPM031_annual', 'OPM031_whole_dataset']
['OPM026_to2038', 'OPM031_whole_dataset']
['OPM026_to2008', 'OPM0263_whole_dataset']
['after_1984', 'whole_dataset']
['no_mar_oct_all_sims_extrap_std']
['OPM031_5p5', 'Christoph_v2_wh

## **Produce metrics for simulations which have already have NNs applied**

In [5]:
class Analysis_Results:
    def __init__(self, NN, applied, RMSE, R2, RMSE_ymb, R2_ymb, RMSE_ym, R2_ym):
        self.NN =  NN                 # Neural network trained 
        self.applied = applied        # Simulation applied to     
        self.RMSE = RMSE              # RMSE (pointwise)
        self.R2 = R2                  # R2 (pointwise)
        self.RMSE_ymb = RMSE_ymb          # RMSE (Integrated by basin)
        self.R2_ymb = R2_ymb              # R2 (Integrated by basin)
        self.RMSE_ym = RMSE_ym           # RMSE (Total Integrated melt)
        self.R2_ym = R2_ym               # R2 (Total Integrated melt)

def RMSE_R2(ref, pred):
    RMSE = np.sqrt(np.mean((ref - pred)**2))
    R2 = regression_analysis(ref, pred)[2]
    return twosigfig(RMSE), twosigfig(R2)

def RMSE_R2_chris(ref, pred):
    RMSE = np.sqrt(np.mean((ref + pred)**2))
    R2 = regression_analysis(ref, -pred)[2]
    return twosigfig(RMSE), twosigfig(R2)

def metrics_nn(this_collection_NN, this_collection_apply, path_model,
                    exp_name = 'slope_front', mod_size = 'small', TS_opt = 'extrap', norm_method = 'std',
                    keep_all_runs = True, annual_only = False, annual_f = '', quick_check = False,
                    verbose = 0):
    """ If a trained neural network has already been applied to a dataset, """
    """ then this function will load in the results for that dataset       """
    if keep_all_runs == False:
        sims = 'just_mean'
    else:
        sims = 'all_sims'
    if annual_only == False:
        ano = ''
    else:
        ano = '_annual'
    ano = ano + '_special_lc_' + this_collection_apply
    fp_ref_pred = path_model + 'applied_nn' + '_' + mod_size + '_' + exp_name + '_' + this_collection_NN + '_' + annual_f + \
                                            sims + '_' + TS_opt + '_' + norm_method + ano + '.csv'
    if quick_check == True:
        df_ref_pred = pd.read_csv(fp_ref_pred, nrows = 5)
    else:
        df_ref_pred = pd.read_csv(fp_ref_pred)
    if verbose == 1:
        print('\033[1m' + 'You have loaded the results of applying the neural network to the test dataset:' + '\033[0m')
        print(fp_ref_pred)
        if quick_check == True:
            print('Warning, only first 5 rows loaded')
    if this_collection_NN == 'Christoph' and this_collection_apply == 'Christoph_v2_whole_dataset':
        RMSE, R2 = RMSE_R2(df_ref_pred.melt_m_ice_per_y*31536000/917, df_ref_pred.melt_pred_mean*31536000/917)
    elif this_collection_NN == 'Christoph' or this_collection_apply == 'Christoph_v2_whole_dataset':
        RMSE, R2 = RMSE_R2_chris(df_ref_pred.melt_m_ice_per_y*31536000/917, df_ref_pred.melt_pred_mean*31536000/917)
    else:
        RMSE, R2 = RMSE_R2(df_ref_pred.melt_m_ice_per_y*31536000/917, df_ref_pred.melt_pred_mean*31536000/917)
    df_ref_pred.loc[:,'melt_Gt_per_y'] = ((df_ref_pred.loc[:,'melt_m_ice_per_y'] * df_ref_pred.loc[:,'area']* 31536000) * 1/1e12)
    df_ref_pred.loc[:,'melt_pred_Gt'] = ((df_ref_pred.loc[:,'melt_pred_mean'] * df_ref_pred.loc[:,'area'] * 31536000) * 1/1e12)
    if this_collection_apply == 'OPM026_whole_dataset':
        df_ref_pred = df_ref_pred[df_ref_pred.year > 1983]
    if verbose == 1:
        print('Calculated melt in Gt per yr')
    df_ymb_mean = df_ref_pred.groupby(['year','month','basin'], as_index = False).mean()
    df_ymb_sum = df_ref_pred.groupby(['year','month','basin'], as_index = False).sum()
    if len(df_ymb_sum) == 0:
        df_ymb_mean = df_ref_pred.groupby(['year','basin'], as_index = False).mean()
        df_ymb_sum = df_ref_pred.groupby(['year','basin'], as_index = False).sum()
    if this_collection_NN == 'Christoph' and this_collection_apply == 'Christoph_v2_whole_dataset':
        RMSE_ymb, R2_ymb = RMSE_R2(df_ymb_sum.melt_Gt_per_y, df_ymb_sum.melt_pred_Gt)
    elif this_collection_NN == 'Christoph' or this_collection_apply == 'Christoph_v2_whole_dataset':
        RMSE_ymb, R2_ymb = RMSE_R2_chris(df_ymb_sum.melt_Gt_per_y, df_ymb_sum.melt_pred_Gt)
    else:
        RMSE_ymb, R2_ymb = RMSE_R2(df_ymb_sum.melt_Gt_per_y, df_ymb_sum.melt_pred_Gt)
    df_ym_mean = df_ref_pred.groupby(['year','month'], as_index = False).mean()
    df_ym_sum = df_ref_pred.groupby(['year','month'], as_index = False).sum()
    if len(df_ym_sum) == 0:
        df_ym_mean = df_ref_pred.groupby(['year'], as_index = False).mean()
        df_ym_sum = df_ref_pred.groupby(['year'], as_index = False).sum()
    if this_collection_NN == 'Christoph' and this_collection_apply == 'Christoph_v2_whole_dataset':
        RMSE_ym, R2_ym = RMSE_R2(df_ym_sum.melt_Gt_per_y, df_ym_sum.melt_pred_Gt)
    elif this_collection_NN == 'Christoph' or this_collection_apply == 'Christoph_v2_whole_dataset':
        RMSE_ym, R2_ym = RMSE_R2_chris(df_ym_sum.melt_Gt_per_y, df_ym_sum.melt_pred_Gt)
    else:
        RMSE_ym, R2_ym = RMSE_R2(df_ym_sum.melt_Gt_per_y, df_ym_sum.melt_pred_Gt)
    if verbose == 1:
        print('Calculated integrated volumes')        
    metrics = Analysis_Results(this_collection_NN, this_collection_apply, RMSE, R2, RMSE_ymb, R2_ymb, RMSE_ym, R2_ym)
    return metrics

In [11]:
recalculate = False

In [12]:
sims = ['OPM026', 'OPM0263', 'OPM031', 'Christoph']
datas = ['OPM026', 'OPM0263', 'OPM031', 'Christoph_v2']
if recalculate == True:
    for i in range(len(sims)):
        try:
            %time metrics = metrics_nn(sims[i], datas[i]+'_whole_dataset', path_model, quick_check = False, verbose = 1)
            if i == 0:
                df_total = pd.DataFrame([vars(metrics)])
            else:
                df = pd.DataFrame([vars(metrics)])
                df_total = pd.concat([df_total, df], axis = 0)
        except:
            print('Failed with', sims[i])
    df_total.to_csv(fp_fdata + 'metrics_validation.csv', index=False)
else:
    df_total = pd.read_csv(fp_fdata + 'metrics_validation.csv')
df_total

,NN,applied,RMSE,R2,RMSE_ymb,R2_ymb,RMSE_ym,R2_ym
0,OPM026,OPM026_whole_dataset,0.93,0.96,1.9,0.99,33,0.92
1,OPM0263,OPM0263_whole_dataset,0.77,0.94,2.5,0.98,80,0.93
2,OPM031,OPM031_whole_dataset,7.50,0.85,21.0,0.98,390,0.96
3,Christoph,Christoph_v2_whole_dataset,1.70,0.95,3.5,0.98,67,0.96


In [13]:
sims = ['OPM026', 'OPM0263', 'OPM031', 'Christoph']
datas = ['OPM026', 'OPM0263', 'OPM031', 'Christoph_v2']
if recalculate == True:
    for i in range(len(sims)):
        for j in range(len(datas)):
            try:
                %time metrics = metrics_nn(sims[i], datas[j]+'_whole_dataset', path_model, quick_check = False, verbose = 0)
                if i == 0:
                    df_total = pd.DataFrame([vars(metrics)])
                else:
                    df = pd.DataFrame([vars(metrics)])
                    df_total = pd.concat([df_total, df], axis = 0)
            except:
                print('Failed with', sims[i], datas[j])
    df_total.to_csv(fp_fdata + 'metrics_testing.csv', index=False)
else:
    df_total = pd.read_csv(fp_fdata + 'metrics_testing.csv')
df_total

,NN,applied,RMSE,R2,RMSE_ymb,R2_ymb,RMSE_ym,R2_ym
0,OPM026,Christoph_v2_whole_dataset,4.20,0.66,8.1,0.91,110,0.93
1,OPM0263,OPM026_whole_dataset,2.60,0.67,3.1,0.98,39,0.89
2,OPM0263,OPM0263_whole_dataset,0.77,0.94,2.5,0.98,80,0.93
3,OPM0263,OPM031_whole_dataset,14.00,0.35,48.0,0.86,1300,0.83
4,OPM0263,Christoph_v2_whole_dataset,5.50,0.45,9.9,0.88,210,0.93
5,OPM031,OPM026_whole_dataset,2.80,0.61,8.2,0.93,100,0.83
6,OPM031,OPM0263_whole_dataset,10.00,0.01,53.0,0.05,1000,0.84
7,OPM031,OPM031_whole_dataset,7.50,0.85,21.0,0.98,390,0.96
8,OPM031,Christoph_v2_whole_dataset,5.10,0.51,10.0,0.86,97,0.92
9,Christoph,OPM026_whole_dataset,1.80,0.85,5.2,0.94,56,0.90


In [15]:
sims = ['OPM026', 'OPM0263', 'OPM031', 'OPM016', 'OPM018', 'OPM021', 'ctrl94', 'isf94', 'isfru94']
if recalculate == True:
    for i in range(len(sims)):
        try:
            %time metrics = metrics_nn('OPM026_OPM0263_OPM031_OPM016_OPM018_OPM021_ctrl94_isf94_isfru94', \
                                 sims[i]+'_whole_dataset', path_model, quick_check = False, verbose = 0)
            if i == 0:
                df_total = pd.DataFrame([vars(metrics)])
            else:
                df = pd.DataFrame([vars(metrics)])
                df_total = pd.concat([df_total, df], axis = 0)
        except:
            print('Failed with', sims[i])
    df_total.to_csv(fp_fdata + 'metrics_all_sims.csv', index=False)
else:
    df_total = pd.read_csv(fp_fdata + 'metrics_all_sims.csv')
df_total

,NN,applied,RMSE,R2,RMSE_ymb,R2_ymb,RMSE_ym,R2_ym
0,OPM026_OPM0263_OPM031_OPM016_OPM018_OPM021_ctr...,OPM026_whole_dataset,1.5,0.89,3.1,0.98,45,0.87
1,OPM026_OPM0263_OPM031_OPM016_OPM018_OPM021_ctr...,OPM0263_whole_dataset,1.3,0.84,3.6,0.97,130,0.87
2,OPM026_OPM0263_OPM031_OPM016_OPM018_OPM021_ctr...,OPM031_whole_dataset,5.0,0.93,10.0,0.99,300,0.96
3,OPM026_OPM0263_OPM031_OPM016_OPM018_OPM021_ctr...,OPM016_whole_dataset,2.1,0.89,3.6,0.97,18,0.91
4,OPM026_OPM0263_OPM031_OPM016_OPM018_OPM021_ctr...,OPM018_whole_dataset,1.2,0.88,2.6,0.95,66,0.87
5,OPM026_OPM0263_OPM031_OPM016_OPM018_OPM021_ctr...,OPM021_whole_dataset,1.6,0.91,4.2,0.96,22,0.97
6,OPM026_OPM0263_OPM031_OPM016_OPM018_OPM021_ctr...,ctrl94_whole_dataset,2.0,0.83,5.5,0.96,53,0.84
7,OPM026_OPM0263_OPM031_OPM016_OPM018_OPM021_ctr...,isf94_whole_dataset,2.7,0.86,4.9,0.97,65,0.96
8,OPM026_OPM0263_OPM031_OPM016_OPM018_OPM021_ctr...,isfru94_whole_dataset,3.2,0.87,6.3,0.98,230,0.99
